In [1]:
%matplotlib inline
import os
import glob
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pathlib
from matplotlib.lines import Line2D
from PIL import Image
import arff


In [2]:
_METRICS_FILE = './files/benchmark_pool'
_METRIC = 'kappa'
_NUM_CLASSIFIER =  10

directory = './files/statistic_test/'+_METRIC+'/'


datasets_to_remove = ['bankmarketingnorm', 'airlinesnorm']
datasets_to_remove += ['breast', 'carnorm', 'covtype', 'electricity', 'higgs200k', 'kddcup99norm', 'ozone', 'pendigitis', 'pokerhand', 'spambase', 'susy100k']
classifiers_to_remove = ['keras_3_Dilated_Conv', 'keras_parallel_3_Dilated_Conv', 'MOA_SGD', 'MOA_WeightedMajorityAlgorithm']

In [3]:
classifier_labels_full = {
    'keras_parallel_3_Dilated_Conv_pooling':'PCNN',
    'sklearn_MLPClassifier':'MLP',
    'MOA_NaiveBayes':'NB',
    'MOA_SingleClassifierDrift':'SCD',
    'MOA_ASHoeffdingTree':'ASHT',
    'MOA_HoeffdingTree':'HT',
    'MOA_HoeffdingOptionTree':'HT',
    'MOA_LeveragingBag':'LBAG',
    'MOA_RandomHoeffdingTree':'RHT',
    'MOA_OzaBoostAdwin':'BO-AD',
    'MOA_OzaBoost':'BO',
    'sklearn_BernoulliNB':'BNB',
    'MOA_HoeffdingAdaptiveTree':'HAT',
    'MOA_ActiveClassifier':'AC',
    'MOA_OzaBag':'BA',
    'MOA_AdaHoeffdingOptionTree':'ADHT',
    'MOA_OzaBagAdwin':'BA-AD',
    'MOA_OzaBagASHT':'B-ASHT',
    'sklearn_PassiveAggressiveClassifier':'PAC',
    'sklearn_SGDClassifier':'SGD',
    'sklearn_Perceptron':'P',
    'MOA_DecisionStump':'DST',
    'MOA_OCBoost':'OCBO',
    'MOA_SPegasos':'SPEG',
    'MOA_MajorityClass':'MC',
    'MOA_Perceptron':'P'
}

In [4]:
# read data
metrics = pd.read_csv(_METRICS_FILE+'.csv')

# Delete datasets and classifier that we dont want
metrics = metrics[~metrics.classifier.isin(classifiers_to_remove)]
metrics = metrics[~metrics.dataset.isin(datasets_to_remove)]
metrics = metrics[metrics['instances']>1000]

metrics = metrics[metrics.classifier.isin(['MOA_ASHoeffdingTree', 'MOA_HoeffdingTree', 'MOA_LeveragingBag', 'MOA_OzaBoostAdwin', 'MOA_SingleClassifierDrift', 'keras_parallel_3_Dilated_Conv_pooling', 'sklearn_MLPClassifier', 'sklearn_SGDClassifier', 'MOA_NaiveBayes'])]

print(len(metrics.dataset.unique()), "datasets")
metrics.sample(10)

29 datasets


,dataset,classifier,instances,attributes,classes,accuracy,kappa,train_time_mean,test_time_mean,total_time_mean,train_time,test_time,total_time
774,FordA,keras_parallel_3_Dilated_Conv_pooling,4911,500,2,0.915380,0.816424,0.036629,0.012942,0.049570,18.021307,6.367340,18.021307
1494,PhalangesOutlinesCorrect,keras_parallel_3_Dilated_Conv_pooling,2648,80,2,0.766221,0.432813,0.025805,0.013916,0.039720,6.838206,3.687613,6.838206
789,FordB,MOA_NaiveBayes,4446,500,2,0.512929,0.046013,0.000369,0.000919,0.001289,0.164384,0.409006,0.573391
1155,Mallat,MOA_OzaBoostAdwin,2400,1024,8,0.257316,0.216491,0.094228,0.017293,0.111521,22.614750,4.150252,26.765002
1387,NonInvasiveFetalECGThorax2,MOA_LeveragingBag,3765,750,42,0.155091,0.130984,0.593251,0.259336,0.852587,223.655535,97.769678,321.425213
2109,TwoPatterns,MOA_NaiveBayes,5000,128,4,0.480870,0.289709,0.000025,0.000262,0.000287,0.012630,0.130971,0.143602
765,FordA,MOA_OzaBoostAdwin,4921,500,2,0.536843,0.045305,0.017452,0.002874,0.020326,8.603843,1.416907,10.020750
239,ChlorineConcentration,sklearn_SGDClassifier,4297,166,3,0.494261,0.099131,0.000420,0.000079,0.000499,0.180692,0.033798,0.214489
2244,Wafer,keras_parallel_3_Dilated_Conv_pooling,7154,152,2,0.995667,0.970155,0.027400,0.011257,0.038657,19.618655,8.060081,19.618655
1376,NonInvasiveFetalECGThorax1,sklearn_MLPClassifier,3755,750,42,0.434130,0.399091,0.005288,0.000663,0.005951,1.988265,0.249445,2.237710


In [5]:
metrics_table = metrics.pivot(index='dataset',columns='classifier')[_METRIC]
metrics_table = metrics_table.dropna()
metrics_table = metrics_table.reindex(metrics_table.mean().sort_values(ascending=False).index, axis=1)
metrics_table = metrics_table.sort_values(by=list(metrics_table.columns), ascending=False)
metrics_table = metrics_table[metrics_table.columns[0:]]
metrics_table.columns = [classifier_labels_full[clf] for clf in metrics_table.columns]


metrics_table

,PCNN,MLP,NB,SCD,HT,ASHT,LBAG,BO-AD,SGD
dataset,,,,,,,,,
TwoPatterns,0.999947,0.831768,0.289709,0.289709,0.289709,0.289709,0.380146,0.304429,0.614509
CinCECGtorso,0.982306,0.981750,0.237493,0.237493,0.233757,0.233757,0.225028,0.000000,0.221763
TwoLeadECG,0.980299,0.642916,0.545044,0.545044,0.544061,0.544061,0.536781,0.857928,0.559434
Wafer,0.970155,0.990666,0.193772,0.191710,0.356147,0.356147,0.963474,0.959682,0.542139
pendigits,0.969291,0.938365,0.824034,0.783898,0.849582,0.849582,0.867245,0.909211,0.800075
FacesUCR,0.967146,0.722795,0.841938,0.841938,0.777421,0.777421,0.860715,0.873506,0.635126
Mallat,0.964654,0.910839,0.931191,0.931191,0.931191,0.931191,0.204766,0.216491,0.786734
FaceAll,0.959695,0.728645,0.829490,0.829490,0.735665,0.735665,0.849261,0.868891,0.637699
Symbols,0.951277,0.830711,0.889887,0.889887,0.888874,0.888874,0.887084,0.890289,0.709223


In [6]:
file_path = directory+_METRIC+'_table.csv'
if not os.path.exists(directory):
    os.makedirs(directory)
metrics_table.to_csv(file_path, sep=',', encoding='utf-8')

In [7]:
with open(file_path, "r+") as f:
    lines = f.readlines()
    #del lines[2]
    #del lines[0]
    lines[0] = lines[0].replace('dataset',' ', 1 )
    f.seek(0)
    f.truncate()
    f.writelines(lines)

In [8]:
[c for c in metrics_table.columns]

['PCNN', 'MLP', 'NB', 'SCD', 'HT', 'ASHT', 'LBAG', 'BO-AD', 'SGD']